# mount drive

In [ ]:
import os
import sys
from google.colab import drive
drive.mount('/content/drive')
path = "/content/drive/My Drive"

os.chdir(path)
os.listdir(path)
sys.path.append("/content/drive/")

In [ ]:
# prepare SpareInst code
!pwd
os.chdir('/content/')
!git clone https://github.com/hustvl/SparseInst.git

sys.path.append("/content/SparseInst/sparseinst")

In [ ]:
# prepare model
# upload from your local or load from your google drive
!cp '/content/drive/My Drive/SparseInst_Colab/sparse_inst_r50vd_giam_aug_8bc5b3.pth' '/content/SparseInst'

In [ ]:
# unzip coco val2017 dataset, this will cost a little time
# or you can use your own images
!pwd
!unzip -uq '/content/drive/My Drive/data/coco/val2017.zip' -d '/content'

# 安装环境

In [ ]:
import os
# change to workdirectory
os.chdir('/content/SparseInst')
# %cd /content/drive/My Drive/AlexNet
!pwd
!ls

In [ ]:
!pip install timm
!pip install pyyaml==5.1

**Install detectron2--v0.3**

In [ ]:
!pwd
!git clone https://github.com/facebookresearch/detectron2.git

In [ ]:
os.chdir('/content/SparseInst/detectron2')
!git checkout tags/v0.3

In [ ]:
# import detectron2
# print(detectron2.version)

In [ ]:
# ATTENTION HERE !!!

# Before run the code, please modify a line of code in detetron2
# See issue#18 in hustvl/SparseInst
# link: https://github.com/hustvl/SparseInst/issues/18

# Then run setup.py, otherwise there will be error occured.

# This bug will be fixed soon

# build detectron2
os.chdir('/content/SparseInst/detectron2')
!python setup.py build develop

sys.path.append("/content/SparseInst/detectron2/")

In [ ]:
# remove built detectron2
# os.chdir('/content/SparseInst/detectron2')
# !rm -rf build/ **/*.so

# Fix package bug

In [ ]:
# The following operartion may be much confusing, but
# there will be some "import errors" occur if not do this

# 1. reinstall fvcore
!pip uninstall fvcore
!pip install fvcore

In [ ]:
# 2. reinstall iopath
!pip uninstall iopath
!pip install iopath

In [ ]:
# 3. reinstall portalocker
!pip uninstall portalocker
!pip install portalocker

In [ ]:
# 4. reinstall yacs
!pip uninstall yacs
!pip install yacs

In [ ]:
# !pip list fvcore
# os.chdir('/content/SparseInst')
import fvcore

In [ ]:
# os.chdir('/content/SparseInst/detectron2')
# !rm -rf build/ **/*.so

# SparseInst Demo

In [ ]:
!nvidia-smi

### Run demo with command line format


In [ ]:
os.chdir('/content/SparseInst')
!python demo.py --config-file configs/sparse_inst_r50vd_giam_aug.yaml --input ../_val2017_part/* --output results --opt MODEL.WEIGHTS sparse_inst_r50vd_giam_aug_8bc5b3.pth INPUT.MIN_SIZE_TEST 512

In [ ]:
os.environ["PATH"] = '/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin'

In [ ]:
os.chdir('/content/SparseInst')
import fvcore

In [ ]:
print(sys.path)
import numpy as np

In [ ]:
# The packs below may raise 'import error'
# uninstall then install may solve the problem but I haven't found the reason

# import fvcore
# import iopath
# import portalocker
# import yacs

### Run with cell

In [ ]:
os.chdir('/content/SparseInst')

# import libs
import argparse
import glob
import multiprocessing as mp
import os
import time
import cv2
import tqdm
from skimage import io

from detectron2.config import get_cfg
from detectron2.data.detection_utils import read_image
from detectron2.utils.logger import setup_logger

from sparseinst import VisualizationDemo, add_sparse_inst_config

In [ ]:
WINDOW_NAME = "COCO detections -- val2017"
def proc_single(image_path, args, cfg, demo):
    # cfg = setup_cfg(args)

    # demo = VisualizationDemo(cfg)

    image_path = './val2017/000000003934.jpg'
    args.input = glob.glob(os.path.expanduser(image_path))

    start_time = time.time()
    img = read_image(path, format="RGB")
    predictions, visualized_output = demo.run_on_image(
        img, args.confidence_threshold)
    print(
        "{}: {} in {:.2f}s".format(
            path,
            "detected {} instances".format(
                len(predictions["instances"]))
            if "instances" in predictions
            else "finished",
            time.time() - start_time,
        )
    )

    if args.output:
        if os.path.isdir(args.output):
            assert os.path.isdir(args.output), args.output
            out_filename = os.path.join(
                args.output, os.path.basename(path))
        else:
            assert len(
                args.output) > 0, "Please specify a directory with args.output"
            out_filename = args.output
        visualized_output.save(out_filename)
    else:
        cv2.namedWindow(WINDOW_NAME, cv2.WINDOW_NORMAL)

In [ ]:
class parser():
    def __init__(self, image_path, output_dir='/content/SparseInst/results/'):
        # reset to your own model-config-file path
        self.config_file = '/content/SparseInst/configs/sparse_inst_r50vd_giam_aug.yaml'

        self.opts = ["MODEL.WEIGHTS", 
                     # reset to your own model.pth path
                     "/content/SparseInst/sparse_inst_r50vd_giam_aug_8bc5b3.pth",
                     "INPUT.MIN_SIZE_TEST", 
                     512]

        self.input = image_path
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        self.output = output_dir

        # reset to your desired threshold
        self.confidence_threshold = 0.3


def _setup_cfg(args):
    cfg = get_cfg()
    add_sparse_inst_config(cfg)

    cfg.merge_from_file(args.config_file)
    cfg.merge_from_list(args.opts)

    cfg.MODEL.RETINANET.SCORE_THRESH_TEST = args.confidence_threshold
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = args.confidence_threshold
    cfg.MODEL.PANOPTIC_FPN.COMBINE.INSTANCES_CONFIDENCE_THRESH = args.confidence_threshold
    cfg.freeze()
    return cfg

args = parser('/content/val2017/000000003934.jpg')
cfg = _setup_cfg(args)


demo = VisualizationDemo(cfg)

In [ ]:
# ATTENTION HERE!
# Before run the code, please modify 2 lines of code in detetron2
# See issue#18, issue#10, issue#13 in hustvl/SparseInst
# link: https://github.com/hustvl/SparseInst/issues/18

# This bug will be fixed soon

In [ ]:
# run the cell
# change the image path to your own
image_path = "/content/drive/MyDrive/SparseInst_Colab/569021651119418_.pic_hd.jpg"
args.input = image_path

img = read_image(image_path, format="RGB")
start_time = time.time()
predictions, visualized_output = demo.run_on_image(
                                img, args.confidence_threshold)

print("\n{}: {} in {:.2f}s".format(image_path,
                                "detected {} instances".format(
                                    len(predictions["instances"]))
                                    if "instances" in predictions
                                    else "finished",
                                time.time() - start_time,
                                ))
out_filename = os.path.join(
                        args.output, os.path.basename(image_path))
print(out_filename)
visualized_output.save(out_filename)

In [ ]:
# run the cell
# change the image url to your own
img_src = "http://n1image.hjfile.cn/shetuan/2017-05-17-1495016837-986-732.jpg"
url_img = io.imread(img_src)

start_time = time.time()
predictions, visualized_output = demo.run_on_image(
                                url_img, args.confidence_threshold)

print("\n{}: {} in {:.2f}s".format(img_src,
                                "detected {} instances".format(
                                    len(predictions["instances"]))
                                    if "instances" in predictions
                                    else "finished",
                                time.time() - start_time,
                                ))
out_filename = os.path.join(
                        args.output, os.path.basename(img_src))
print(out_filename)
visualized_output.save(out_filename)